In [1]:
import mxnet as mx
import gluonnlp as nlp

import numpy as np
import os
import random
import sacremoses
import time
from tqdm.notebook import tqdm as tqdm

# Local Libraries
import nmt
import dataprocessor
import utils
import nmt.transformer_hparams

# Seeds for reproducibility
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)

# CPU setup
# ctx = mx.cpu()
# Single GPU setup
ctx = mx.gpu(0)

[nltk_data] Downloading package punkt to /home/andreto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/andreto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/andreto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# IWSLT2015 Dataset (Train, Validation and Test)

# Dataset Parameters
src_lang, tgt_lang = "vi", "en"
src_max_len, tgt_max_len = 50, 50

iwslt_train_text = nlp.data.IWSLT2015("train",
                                      src_lang=src_lang,
                                      tgt_lang=tgt_lang)

iwslt_val_text   = nlp.data.IWSLT2015("val",
                                      src_lang=src_lang,
                                      tgt_lang=tgt_lang)

iwslt_test_text  = nlp.data.IWSLT2015("test",
                                      src_lang=src_lang,
                                      tgt_lang=tgt_lang)


iwslt_src_vocab = iwslt_train_text.src_vocab
iwslt_tgt_vocab = iwslt_train_text.tgt_vocab

/home/andreto/.local/lib/python3.7/site-packages/gluonnlp/vocab/vocab.py:591: UserWarning: Detected a corrupted index in the deserialize vocabulary. For versions before GluonNLP v0.7 the index is corrupted by specifying the same token for different special purposes, for example eos_token == padding_token. Deserializing the vocabulary nevertheless.
  'Detected a corrupted index in the deserialize vocabulary. '


In [3]:
print("Length of train set:", len(iwslt_train_text))
print("Length of val set  :", len(iwslt_val_text))
print("Length of test set :", len(iwslt_test_text))

Length of train set: 133166
Length of val set  : 1553
Length of test set : 1268


In [4]:
# Dataset processing: clipping, tokenizing, indexing and adding of EOS (src/tgt) / BOS (tgt)
iwslt_train_processed = iwslt_train_text.transform(
    dataprocessor.TrainValDataTransform(
        iwslt_src_vocab,
        iwslt_tgt_vocab,
        src_max_len,
        tgt_max_len),
    lazy=False)

iwslt_val_processed   = iwslt_val_text.transform(
    dataprocessor.TrainValDataTransform(
        iwslt_src_vocab,
        iwslt_tgt_vocab,
        src_max_len,
        tgt_max_len),
    lazy=False)

iwslt_test_processed  = iwslt_test_text.transform(
    dataprocessor.TrainValDataTransform(
        iwslt_src_vocab,
        iwslt_tgt_vocab,
        src_max_len,
        tgt_max_len),
    lazy=False)

In [5]:
# Target Sequences (Val, Test)
fetch_tgt_sentence = lambda src, tgt: tgt.split()
val_tgt_sentences = list(iwslt_val_text.transform(fetch_tgt_sentence))
test_tgt_sentences = list(iwslt_test_text.transform(fetch_tgt_sentence))

In [6]:
# Create Gluon Datasets
# Not needed for training, as training data will be sharded later
iwslt_train_transformed = iwslt_train_processed.transform(
    lambda src, tgt: (src, tgt, len(src), len(tgt)),
    lazy=False)

iwslt_val_dataset = mx.gluon.data.SimpleDataset(
    [(ele[0], ele[1], len(ele[0]), len(ele[1]),i) for i, ele in enumerate(iwslt_val_processed)])

iwslt_test_dataset = mx.gluon.data.SimpleDataset(
    [(ele[0], ele[1], len(ele[0]), len(ele[1]), i) for i, ele in enumerate(iwslt_test_processed)])

In [7]:
# Hyperparameters for Dataloaders and Training
hparams = nmt.transformer_hparams

In [8]:
hparams.num_hidden = 1024
hparams.num_layers = 2
hparams.dropout = 0.2
hparams.num_buckets = 5
hparams.lr = 0.0002
#hparams.lr = 0.0003 achieves 21.44 test_bleu: qualitative evaluation didn't work
#hparams.lr = 0.0001 achieves 19.66 test_bleu: qualitative evaluation worked
hparams.clip = 5
hparams.epochs = 12

In [9]:
# Create Gluon Samplers and DataLoaders

# Helper function for lengths
def get_data_lengths(dataset):
    get_lengths = lambda *args: (args[2], args[3])
    return list(dataset.transform(get_lengths))

# Bucket scheme
bucket_scheme = nlp.data.ExpWidthBucket(bucket_len_step=1.2)

iwslt_train_lengths = get_data_lengths(iwslt_train_transformed)
iwslt_val_lengths = get_data_lengths(iwslt_val_dataset)
iwslt_test_lengths = get_data_lengths(iwslt_test_dataset)

train_batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Pad(),
    nlp.data.batchify.Pad(),
    nlp.data.batchify.Stack(dtype='float32'),
    nlp.data.batchify.Stack(dtype='float32'))

test_batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Pad(),
    nlp.data.batchify.Pad(),
    nlp.data.batchify.Stack(dtype='float32'),
    nlp.data.batchify.Stack(dtype='float32'),
    nlp.data.batchify.Stack())

target_val_lengths = list(map(lambda x: x[-1], iwslt_val_lengths))
target_test_lengths = list(map(lambda x: x[-1], iwslt_test_lengths))

train_batch_sampler = nlp.data.FixedBucketSampler(
    lengths=iwslt_train_lengths,
    batch_size=hparams.batch_size,
    num_buckets=hparams.num_buckets,
    ratio=0,
    shuffle=True,
    use_average_length=False,
    num_shards=0,
    bucket_scheme=bucket_scheme)
    
train_data_loader = nlp.data.ShardedDataLoader(
    iwslt_train_transformed,
    batch_sampler=train_batch_sampler,
    batchify_fn=train_batchify_fn,
    num_workers=8)

val_batch_sampler = nlp.data.FixedBucketSampler(
    lengths=iwslt_val_lengths,
    batch_size=hparams.test_batch_size,
    num_buckets=hparams.num_buckets,
    ratio=0,
    shuffle=False,
    use_average_length=False,
    bucket_scheme=bucket_scheme)

val_data_loader = mx.gluon.data.DataLoader(
    iwslt_val_dataset,
    batch_sampler=val_batch_sampler,
    batchify_fn=test_batchify_fn,
    num_workers=8)

test_batch_sampler = nlp.data.FixedBucketSampler(
    lengths=iwslt_test_lengths,
    batch_size=hparams.test_batch_size,
    num_buckets=hparams.num_buckets,
    ratio=0,
    shuffle=False,
    use_average_length=False,
    bucket_scheme=bucket_scheme)

test_data_loader = mx.gluon.data.DataLoader(
    iwslt_test_dataset,
    batch_sampler=test_batch_sampler,
    batchify_fn=test_batchify_fn,
    num_workers=8)

/home/andreto/.local/lib/python3.7/site-packages/gluonnlp/data/batchify/batchify.py:235: UserWarning: Padding value is not given and will be set automatically to 0 in data.batchify.Pad(). Please check whether this is intended (e.g. value of padding index in the vocabulary).
  'Padding value is not given and will be set automatically to 0 '


In [10]:
# Transformer Model
transformer_encoder, transformer_decoder, transformer_one_step_ahead_decoder = nlp.model.transformer.get_transformer_encoder_decoder(
    hidden_size=hparams.num_hidden,
    dropout=hparams.dropout,
    num_layers=hparams.num_layers)

transformer_model = nlp.model.translation.NMTModel(
    src_vocab=iwslt_src_vocab,
    tgt_vocab=iwslt_tgt_vocab,
    encoder=transformer_encoder,
    decoder=transformer_decoder,
    one_step_ahead_decoder=transformer_one_step_ahead_decoder,
    #embed_size=hparams.num_hidden,
    embed_size=hparams.num_units,
    prefix='transformer_')

transformer_model.initialize(init=mx.init.Xavier(magnitude=1.0), ctx=ctx)
static_alloc = True
transformer_model.hybridize(static_alloc=static_alloc)

In [11]:
# Translator (using model defined above)
transformer_translator = nmt.translation.BeamSearchTranslator(
    model=transformer_model,
    beam_size=hparams.beam_size,
    scorer=nlp.model.BeamSearchScorer(
        alpha=hparams.lp_alpha,
        K=hparams.lp_k),
    max_length=150)

In [12]:
# Loss function
loss_function = nlp.loss.MaskedSoftmaxCELoss()
loss_function.hybridize(static_alloc=static_alloc)

In [13]:
# Evaluation function (used also on training loop for validation)
def evaluate(
    data_loader,
    model,
    translator,
    loss_function,
    tgt_vocab):

    translation_out = []
    all_inst_ids = []
    avg_loss_denom = 0
    avg_loss = 0.0
    
    for _, (src_seq, tgt_seq, src_valid_length, tgt_valid_length, inst_ids) in enumerate(tqdm(data_loader)):
        
        src_seq = src_seq.as_in_context(ctx)
        tgt_seq = tgt_seq.as_in_context(ctx)
        src_valid_length = src_valid_length.as_in_context(ctx)
        tgt_valid_length = tgt_valid_length.as_in_context(ctx)
        
        # Calculating Loss
        out, _ = model(
            src_seq,
            tgt_seq[:, :-1],
            src_valid_length,
            tgt_valid_length - 1)

        loss = loss_function(
            out,
            tgt_seq[:, 1:],
            tgt_valid_length - 1).sum().asscalar()
        
        all_inst_ids.extend(inst_ids.asnumpy().astype(np.int32).tolist())
        avg_loss += loss * (tgt_seq.shape[1] - 1)
        avg_loss_denom += (tgt_valid_length - 1).sum().asscalar()
        
        # Translate
        samples, _, sample_valid_length = translator.translate(
            src_seq=src_seq,
            src_valid_length=src_valid_length)
        
        max_score_sample = samples[:, 0, :].asnumpy()
        sample_valid_length = sample_valid_length[:, 0].asnumpy()
        
        for i in range(max_score_sample.shape[0]):
            translation_out.append(
                [tgt_vocab.idx_to_token[ele] for ele in
                 max_score_sample[i][1:(sample_valid_length[i] - 1)]])
    
    avg_loss = avg_loss / avg_loss_denom
    real_translation_out = [None for _ in range(len(all_inst_ids))]
    
    for ind, sentence in zip(all_inst_ids, translation_out):
        real_translation_out[ind] = sentence
    
    return avg_loss, real_translation_out

In [14]:
# Let's train
trainer = mx.gluon.Trainer(transformer_model.collect_params(), hparams.optimizer, {'learning_rate': hparams.lr})

best_valid_bleu = 0.0

for epoch_id in tqdm(range(hparams.epochs)):

    log_loss = 0
    log_denom = 0
    log_avg_gnorm = 0
    log_wc = 0
    log_start_time = time.time()
    
    for batch_id, (src_seq, tgt_seq, src_valid_length, tgt_valid_length) in enumerate(tqdm(train_data_loader)):
        
        src_seq = src_seq.as_in_context(ctx)
        tgt_seq = tgt_seq.as_in_context(ctx)
        src_valid_length = src_valid_length.as_in_context(ctx)
        tgt_valid_length = tgt_valid_length.as_in_context(ctx)
        
        with mx.autograd.record():
            out, _ = transformer_model(
                src_seq,
                tgt_seq[:, :-1],
                src_valid_length,
                tgt_valid_length - 1)

            loss = loss_function(out, tgt_seq[:, 1:], tgt_valid_length - 1).mean()
            loss = loss * (tgt_seq.shape[1] - 1)
            log_loss += loss * tgt_seq.shape[0]
            log_denom += (tgt_valid_length - 1).sum()
            loss = loss / (tgt_valid_length - 1).mean()
            loss.backward()
        
        grads = [p.grad(ctx) for p in transformer_model.collect_params().values() if p.grad_req != "null"]
        gnorm = mx.gluon.utils.clip_global_norm(grads, hparams.clip)
        trainer.step(1)
        
        src_wc = src_valid_length.sum().asscalar()
        tgt_wc = (tgt_valid_length - 1).sum().asscalar()
        log_loss = log_loss.asscalar()
        log_denom = log_denom.asscalar()
        log_avg_gnorm += gnorm
        log_wc += src_wc + tgt_wc
        
        if (batch_id + 1) % hparams.log_interval == 0:
            wps = log_wc / (time.time() - log_start_time)
            print("[Epoch {} Batch {}/{}] loss={:.4f}, ppl={:.4f}, gnorm={:.4f}, "
                         "throughput={:.2f}K wps, wc={:.2f}K"
                         .format(epoch_id, batch_id + 1, len(train_data_loader),
                                 log_loss / log_denom,
                                 np.exp(log_loss / log_denom),
                                 log_avg_gnorm / hparams.log_interval,
                                 wps / 1000, log_wc / 1000))
            
            log_start_time = time.time()
            log_loss = 0
            log_denom = 0
            log_avg_gnorm = 0
            log_wc = 0
    
    valid_loss, valid_translation_out = evaluate(
        val_data_loader,
        transformer_model,
        transformer_translator,
        loss_function,
        iwslt_tgt_vocab)

    valid_bleu_score, _, _, _, _ = nmt.bleu.compute_bleu([val_tgt_sentences], valid_translation_out)
    print("[Epoch {}] valid Loss={:.4f}, valid ppl={:.4f}, valid bleu={:.2f}"
          .format(epoch_id, valid_loss, np.exp(valid_loss), valid_bleu_score * 100))

    if valid_bleu_score > best_valid_bleu:
        best_valid_bleu = valid_bleu_score
        print("Save best parameters to {}".format(hparams.file_name))
        transformer_model.save_parameters(hparams.file_name)
    
    if epoch_id + 1 >= (hparams.epochs * 2) // 3:
        new_lr = trainer.learning_rate * hparams.lr_update_factor
        print("Learning rate change to {}".format(new_lr))
        trainer.set_learning_rate(new_lr)

if os.path.exists(hparams.file_name):
    transformer_model.load_parameters(hparams.file_name)

valid_loss, valid_translation_out = evaluate(
    val_data_loader,
    transformer_model,
    transformer_translator,
    loss_function,
    iwslt_tgt_vocab)

valid_bleu_score, _, _, _, _ = nmt.bleu.compute_bleu([val_tgt_sentences], valid_translation_out)
print("Best model valid Loss={:.4f}, valid ppl={:.4f}, valid bleu={:.2f}"
      .format(valid_loss, np.exp(valid_loss), valid_bleu_score * 100))

test_loss, test_translation_out = evaluate(
    test_data_loader,
    transformer_model,
    transformer_translator,
    loss_function,
    iwslt_tgt_vocab)

test_bleu_score, _, _, _, _ = nmt.bleu.compute_bleu([test_tgt_sentences], test_translation_out)
print("Best model test Loss={:.4f}, test ppl={:.4f}, test bleu={:.2f}'"
      .format(test_loss, np.exp(test_loss), test_bleu_score * 100))

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/1043 [00:00<?, ?it/s]

Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/mxnet_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2022-06-19 14:06:32.626 ip-172-31-28-47:4312 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-06-19 14:06:32.654 ip-172-31-28-47:4312 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
[Epoch 0 Batch 100/1043] loss=6.7084, ppl=819.2355, gnorm=1.8441, throughput=66.23K wps, wc=606.57K
[Epoch 0 Batch 200/1043] loss=5.4545, ppl=233.7993, gnorm=0.8658, throughput=78.02K wps, wc=584.18K
[Epoch 0 Batch 300/1043] loss=5.0439, ppl=155.0811, gnorm=0.7771, throughput=79.05K wps, wc=592.78K
[Epoch 0 Batch 400/1043] loss=4.7569, ppl=116.3892, gnorm=0.7737, throughp

  0%|          | 0/27 [00:00<?, ?it/s]

[Epoch 0] valid Loss=3.6623, valid ppl=38.9501, valid bleu=8.60
Save best parameters to transformer_vi_en_512.params


...

[Epoch 11] valid Loss=2.3541, valid ppl=10.5284, valid bleu=20.95
Learning rate change to 6.25e-06


  0%|          | 0/27 [00:00<?, ?it/s]

Best model valid Loss=2.3550, valid ppl=10.5381, valid bleu=21.05


  0%|          | 0/23 [00:00<?, ?it/s]

Best model test Loss=2.2326, test ppl=9.3240, test bleu=22.87'


In [18]:
print("Qualitative Evaluation: Translating from Vietnamese to English")

expected_tgt_seq = "I like to read books."
print("Expected translation:")
print(expected_tgt_seq)
# From Google Translate
src_seq = "Tôi thích đọc sách kỹ thuật."
print("In Vietnamese (from Google Translate):")
print(src_seq)

src_sentence = iwslt_src_vocab[src_seq.split()]
src_sentence.append(iwslt_src_vocab[iwslt_src_vocab.eos_token])
src_npy = np.array(src_sentence, dtype=np.int32)
src_nd = mx.nd.array(src_npy)
src_nd = src_nd.reshape((1, -1)).as_in_context(ctx)
src_valid_length = mx.nd.array([src_nd.shape[1]]).as_in_context(ctx)

samples, _, sample_valid_length = transformer_translator.translate(
    src_seq=src_nd,
    src_valid_length=src_valid_length)

max_score_sample = samples[:, 0, :].asnumpy()
sample_valid_length = sample_valid_length[:, 0].asnumpy()

translation_out = []
for i in range(max_score_sample.shape[0]):
    translation_out.append(
        [iwslt_tgt_vocab.idx_to_token[ele] for ele in
         max_score_sample[i][1:(sample_valid_length[i] - 1)]])

print("The English translation is:")
print(" ".join(translation_out[0]))

Qualitative Evaluation: Translating from Vietnamese to English
Expected translation:
I like to read books.
In Vietnamese (from Google Translate):
Tôi thích đọc sách kỹ thuật.
The English translation is:
I love reading books .
